In [1]:
import os
import pandas as pd
import IPython.display as ipd
import libfmp.b
import gc
import logging

In [2]:
path = '/Users/Kittipat/Desktop/fypW/Program/Train_test/Train_songs'  # replace with the path to your directory
files = []

for file_name in os.listdir(path):
    if os.path.isfile(os.path.join(path, file_name)):
        if '.wav' in file_name:
            files.append(file_name)

print(files)

#logging.shutdown()
#os.remove('test.log')

['0_Bm_E_C#m_F#m_Bm_E_A_A_110bpm_คนใจง่าย.wav', '0_C#m_B_C#m_B_C#m_B_A_B_112bpm_ขี้หึง.wav', '0_D_D_C#m_C#m_Bm_E_A_A_110bpm_คนใจง่าย.wav', '0_Em7=0.5_Cmaj7=0.5_Em7=0.5_Cmaj7_D_86bpm_ปล่อย.wav', '0_E_E_A_B_C#m_A_B_B_112bpm_ขี้หึง.wav', '0_E_E_G#m_A_E_E_C#m_B_112bpm_ขี้หึง.wav', '0_Fmaj7_Em7_Dm7=2_G7=2_C=2_C7=2_Fmaj7_Em7=2_Am7=2_Dm7_G_68bpm_ถ้า.wav', '0_Fmaj7_Em7_Dm7=2_G7=2_C_Fmaj7_Em7=2_Am7=2_Dm7=2_G7=2_C_68bpm_ถ้า.wav', '0_G=0.5_Em7=0.5_C=0.5_D=0.5_86bpm_ปล่อย.wav', '0_Gmaj7=0.5_Em7=0.5_Cmaj7=0.5_D=0.5_86bpm_ปล่อย.wav', '1_A_Amaj7_D_D_68bpm_นะหน้าธอง.wav', '1_Cmaj7_D7_G_G_100bpm_หมอกจาง.wav', '1_D_Dmaj7_Em7_A_79bpm_ถ้าเธอรักฉันจริง.wav', '1_E_A_C#m_A=2_B=2_73bpm_ร.wav', '1_G_A_F#m7_Bm_110bpm_คุกเข่า.wav', '1_G_A_F#m7_Bm_110bpm_คุกเข่า_stutter.wav', '1_G_Em7_Cmaj7_D_100bpm_หมอกจาง.wav', '2_C_Am7_Dm7_G7_C_Am7_Dm7_G7_68bpm_ถ้า.wav', '2_C_Em7_Am7_F_C_C7_Fmaj7_G_65bpm_ทุ้มอยู่.wav', '2_Dm7_Em7_Fmaj7_Em7_Dm7_C7_Em7_G_G_65bpm_ทุ้มอยู่.wav', '2_G_D_F#_Em7_D_C_Bm7_Am7_D_76bpm_ปีนี้ไม่ต้องเหงา.w

In [3]:
logging.basicConfig(filename='test.log', level = logging.DEBUG)        # config log
#each_song = []
#new_files = []
for items in files:                                                    # initiate for loop for each file name
    all_chord = items.split('_')                                       # for each file name, split _
    logging.debug(f'all_chord = {all_chord}')
    
    for chord in all_chord:                                            # find bpm and split latter part out
        index = all_chord.index(chord)                                 # index of string of each file names                  
        logging.debug(f'index = {index}')                              
        if 'bpm' in chord:                                             # find bpm
            bpm = int(chord.split('b')[0])
            logging.debug(f'bpm = {bpm}')
        if index != 0:
            if chord[0].isdigit():                                     # split the latter part
                list_len = len(all_chord)
                logging.debug(f'list_len = {list_len}')
                del all_chord[index:list_len]
                logging.debug(f'all_chord after del = {all_chord}')
                break
    
    t = (1/(bpm/60))*4                                                 # time for each chord
    ann = []                                                           # annotation matrix for each files
    if all_chord[0] == '0':                                             # case 0: for the future
        del all_chord[0]
        mat_len = len(all_chord) 
        index = 0
        n = 0
        while True:
            if index == (mat_len):
                logging.info(f'going out, mat_len = {mat_len}')
                break
            if '=' in all_chord[index%mat_len]:
                special_note = all_chord[index].split('=')
                tmp = float(special_note[1])
                tmp = 1/tmp
                ann.append((n*t, (n+tmp)*t, special_note[0]))
                n += tmp
                logging.debug(f'= found, special_note = {special_note}, tmp = {tmp}, ann = {ann}')
            else:
                ann.append((n*t, (n+1)*t, all_chord[index]))
                n += 1
                logging.debug(f'normal length, ann = {ann}')
            index += 1
    elif all_chord[0] == '1':
        del all_chord[0]
        mat_len = len(all_chord) 
        index = 0
        n = 0
        while True:
            if index == (mat_len*2):
                logging.info(f'going out, mat_len = {mat_len}')
                break
            if '=' in all_chord[index%mat_len]:
                special_note = all_chord[index%mat_len].split('=')
                tmp = int(special_note[1])
                tmp = 1/tmp
                ann.append((n*t, (n+tmp)*t, special_note[0]))
                n += tmp
                logging.debug(f'= found, special_note = {special_note}, tmp = {tmp}, ann = {ann}')
            else:
                ann.append((n*t, (n+1)*t, all_chord[index%mat_len]))
                n += 1
                logging.debug(f'normal length, ann = {ann}')
            index += 1
            
    else:
        del all_chord[0]
        mat_len = len(all_chord) 
        index = 0
        n = 0
        while True:
            if index == (mat_len*2):
                logging.info(f'going out, mat_len = {mat_len}')
                break
            if '=' in all_chord[index%mat_len]:
                special_note = all_chord[index%mat_len].split('=')
                tmp = int(special_note[1]) + 2
                tmp = 1/tmp
                ann.append((n*t, (n+tmp)*t, special_note[0]))
                n += tmp
                logging.debug(f'= found, special_note = {special_note}, tmp = {tmp}, ann = {ann}')
            else:
                tmp = 0.5
                ann.append((n*t, (n+tmp)*t, all_chord[index%mat_len]))
                logging.debug(f'normal length, ann = {ann}')
                n += tmp
            index += 1
            
    ann_df = pd.DataFrame(ann, columns=['start', 'end', 'label'])
    path = 'Train_songs/label/' + items.split('.wav')[0] + '.csv'
    print(path)
    ann_test = libfmp.b.b_annotation.write_csv(ann_df, path)
    


Train_songs/label/0_Bm_E_C#m_F#m_Bm_E_A_A_110bpm_คนใจง่าย.csv
Train_songs/label/0_C#m_B_C#m_B_C#m_B_A_B_112bpm_ขี้หึง.csv
Train_songs/label/0_D_D_C#m_C#m_Bm_E_A_A_110bpm_คนใจง่าย.csv
Train_songs/label/0_Em7=0.5_Cmaj7=0.5_Em7=0.5_Cmaj7_D_86bpm_ปล่อย.csv
Train_songs/label/0_E_E_A_B_C#m_A_B_B_112bpm_ขี้หึง.csv
Train_songs/label/0_E_E_G#m_A_E_E_C#m_B_112bpm_ขี้หึง.csv
Train_songs/label/0_Fmaj7_Em7_Dm7=2_G7=2_C=2_C7=2_Fmaj7_Em7=2_Am7=2_Dm7_G_68bpm_ถ้า.csv
Train_songs/label/0_Fmaj7_Em7_Dm7=2_G7=2_C_Fmaj7_Em7=2_Am7=2_Dm7=2_G7=2_C_68bpm_ถ้า.csv
Train_songs/label/0_G=0.5_Em7=0.5_C=0.5_D=0.5_86bpm_ปล่อย.csv
Train_songs/label/0_Gmaj7=0.5_Em7=0.5_Cmaj7=0.5_D=0.5_86bpm_ปล่อย.csv
Train_songs/label/1_A_Amaj7_D_D_68bpm_นะหน้าธอง.csv
Train_songs/label/1_Cmaj7_D7_G_G_100bpm_หมอกจาง.csv
Train_songs/label/1_D_Dmaj7_Em7_A_79bpm_ถ้าเธอรักฉันจริง.csv
Train_songs/label/1_E_A_C#m_A=2_B=2_73bpm_ร.csv
Train_songs/label/1_G_A_F#m7_Bm_110bpm_คุกเข่า.csv
Train_songs/label/1_G_A_F#m7_Bm_110bpm_คุกเข่า_stutter.csv
Tr

In [30]:
handlers = logging.getLogger().handlers[:]
#print(handlers)
for handler in handlers:
    #print(handler)
    logging.getLogger().removeHandler(handler)
    handler.close()
#os.remove('test.log')

[]


In [14]:
print(libfmp.b.b_annotation.read_csv("Train_songs/1_D_Dmaj7_Em7_A_79bpm_ถ้าเธอรักฉันจริง.csv", header=True, add_label=False))
df = libfmp.b.b_annotation.read_csv("Train_songs/1_D_Dmaj7_Em7_A_79bpm_ถ้าเธอรักฉันจริง.csv", header=True, add_label=False)
x = df.at[0,'start']
type(x)

       start        end  label
0   0.000000   3.037975      D
1   3.037975   6.075949  Dmaj7
2   6.075949   9.113924    Em7
3   9.113924  12.151899      A
4  12.151899  15.189873      D
5  15.189873  18.227848  Dmaj7
6  18.227848  21.265823    Em7
7  21.265823  24.303797      A


numpy.float64